# SMS Corpus Preprocessing

Download and clean dataset. Optional as the cleaned dataset is already provided in corpus folder.

In [1]:
import json
import pandas as pd

In [9]:
with open("../data/smsCorpus_en_2015.03.09_all.json") as f:
    sms = json.loads(f.read())

for key, value in sms["smsCorpus"].items() :
    print(key)

@date
@version
message


In [3]:
sms_df = pd.DataFrame(sms["smsCorpus"]["message"])

In [10]:
sms_df.shape
sms_df.head()

,@id,text,source,destination,messageProfile,collectionMethod
0,10120,{'$': 'Bugis oso near wat...'},"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown..."
1,10121,"{'$': 'Go until jurong point, crazy.. Availabl...","{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown..."
2,10122,{'$': 'I dunno until when... Lets go learn pil...,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown..."
3,10123,{'$': 'Den only weekdays got special price... ...,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown..."
4,10124,{'$': 'Meet after lunch la...'},"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown..."


In [11]:
sms_df["user_id"] = sms_df.source.apply(lambda s: s["srcNumber"]["$"])

In [12]:
sms_df.user_id.value_counts()

23249055a638bbc9b1fc5eb7dac9b4259524183451bc74bc                    4951
0                                                                   4754
975bd06869dccbc3b74b999a840f68a537795b6c8b833dfe                    3976
c3e3c793bbdb71d520fa190b5c23b4cf796b52d96b40d9d938cc3db68eb0e909    3383
df501e0b21bc88cc1e464dd95bf722da8865851c93267498                    2621
                                                                    ... 
28                                                                     1
e26a68d382a1a95820819bfdb0836c49                                       1
22                                                                     1
98                                                                     1
50                                                                     1
Name: user_id, Length: 343, dtype: int64

In [13]:
sms_df["text"] = sms_df.text.apply(lambda s: s["$"])

In [14]:
# drop duplicated text
sms_df.drop_duplicates(subset=["text"], inplace=True)

In [15]:
# filter users with at least 10 smses
sms_df = sms_df.groupby("user_id").filter(lambda x: len(x) >= 10)

In [16]:
sms_df.shape

(48092, 7)

In [17]:
# remap user id from 0 to N
user_id_remap = {value:ix for ix, value in enumerate(sms_df.user_id.unique())}

In [18]:
sms_df.user_id.replace(user_id_remap, inplace=True)

In [19]:
sms_df.user_id.nunique()

199

In [21]:
sms_df.shape
sms_df.head()

,@id,text,source,destination,messageProfile,collectionMethod,user_id
0,10120,Bugis oso near wat...,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown...",0
1,10121,"Go until jurong point, crazy.. Available only ...","{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown...",0
2,10122,I dunno until when... Lets go learn pilates...,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown...",0
3,10123,Den only weekdays got special price... Haiz......,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown...",0
4,10124,Meet after lunch la...,"{'srcNumber': {'$': 51}, 'phoneModel': {'@manu...","{'@country': 'unknown', 'destNumber': {'$': 'u...","{'@language': 'en', '@time': 'unknown', '@type...","{'@collector': 'howyijue', '@method': 'unknown...",0


In [22]:
# parallelize this?
for i in sms_df.index:
    sms_df.loc[i].to_json("/Users/pscollins/MACS_30123/final-project-sabrina-phoebe/data/json_files/row{}.json".format(i))

In [16]:
sms_df[["user_id", "text"]].to_csv("../data/sg_sms_corpus_en.csv", index=False)